In [1]:
from pymycobot import MyCobotSocket
import time
import numpy as np
from DHRobotGT import myCobot320
from spatialmath import SE3
import itertools

# mc = MyCobotSocket('10.42.0.1', 9000)
cobot = myCobot320(rotar_base=True)

class RobTarget:
    def __init__(self, pose, config=None):
        """
        Args:
            pose: Matriz 4x4 (tipo numpy o SE3) que representa la pose cartesiana.
            config: Lista con configuración articular, por ejemplo [1, -1, 1].
        """
        # Convertir SE3 a matriz si es necesario
        if isinstance(pose, SE3):
            self.pose = pose
        elif hasattr(pose, "shape") and pose.shape == (4, 4):
            self.pose = SE3(pose)
        else:
            raise ValueError("La pose debe ser una matriz 4x4 o un objeto SE3.")

        if config is not None:
            if isinstance(config, list) and all(isinstance(x, int) for x in config):
                self.config = config
            else:
                raise ValueError("La configuración debe ser una lista como [1, -1, 1].")
        else:
            self.config = [1, 1, 1]  # Default genérico
        
        self.valid_configs = []

    def find_valid_configs(self, robot_model, tool, wobj):
        T_global = wobj * self.pose * tool.inv()
        # Todas las combinaciones posibles de [±1, ±1, ±1]
        configs = list(itertools.product([1, -1], repeat=3))
        
        valid_solutions = []

        for conf in configs:
            solution = robot_model.ikine(T_global, conf)[0]
            if len(solution) > 0:
                self.valid_configs.append(conf)
    
        if self.valid_configs:
            print("Configuraciones válidas:", ", ".join([str(c) for c in self.valid_configs]))
        else:
            print("⚠️ Ninguna configuración resuelve la IK para esta pose.")

    def as_list(self, degrees=True):
        """
        Devuelve la pose como lista [x, y, z, rx, ry, rz]
        En grados por default (para pymycobot).
        """
        pos = self.pose.t
        rpy = self.pose.rpy(order='xyz', unit='deg' if degrees else 'rad')
        return [*pos, *rpy]

    def as_matrix(self):
        """
        Devuelve la pose como matriz homogénea 4x4 (numpy array).
        """
        return self.pose.A

    def get_SE3(self):
        """
        Devuelve la pose como objeto SE3.
        """
        return self.pose

    def offset(self, dx=0, dy=0, dz=0, wobj=SE3()):
        """
        Desplaza el RobTarget en el marco del wobj, sin cambiar orientación ni configuración.
        
        Args
            dx, dy, dz: Desplazamientos en mm respecto al wobj.
            wobj: Objeto SE3 que representa el marco del wobj (default: identidad).

        Returns:
            Nuevo RobTarget desplazado.
        """
        # Vector de desplazamiento en el marco del wobj
        T_offset_local = SE3(dx, dy, dz)

        # Nueva pose desplazada en global
        T_global_offset = wobj * self.pose * T_offset_local

        # Convertimos de nuevo a coordenadas relativas al wobj
        new_pose = wobj.inv() * T_global_offset

        return RobTarget(new_pose, config=self.config.copy())


    def __repr__(self):
        return f"RobTarget(pose=SE3({self.pose.t.tolist()}, rpy={self.pose.rpy(unit='deg').tolist()} deg), config={self.config})"

def matrix_to_pose(T_matrix):
    T = SE3(T_matrix)
    x, y, z = T.t       # Extraer traslación

    # Extraer rotación como RPY (en radianes), orden ZYX
    rx, ry, rz = T.rpy(order='zyx', unit='rad')
    rx, ry, rz = np.rad2deg([rx, ry, rz])   # Convertimos a grados
    return [x, y, z, rx, ry, rz]

def MoveJAngles(q, spd):
    """
    Envía al robot al vector de variables articulares pedido.

    Parameters
    ----------
        q : Array(1,6)
            Vector de variables articulares [rad]
        spd : int (1 - 100)
            Velocidad 
    """
    # mc.send_angles(np.degrees(q).tolist(), spd)

def MoveJoint(robt, spd, tool, wobj):
    # La pose se va a tener que calcular como el robtarget desde el wobj para una tool dada.
    # Calcular pose global: wobj * robtarget * inv(tool)
    pose_calc = wobj * robt.get_SE3() * tool.inv()

    # Convertir a formato [x, y, z, rx, ry, rz] en grados. En realidad no haría falta:
    # vamos a usar ikine para controlar la conf.
    # pose = list(pose_calc.t) + list(pose_calc.rpy(unit='deg'))

    # Usamos la cinemática inversa de la toolbox para tener control de la config.
    # Acá va a haber que agregar un try porque puede llegar a fallar la resolución de ikine.
    q_pose = cobot.ikine(pose_calc, robt.config)[0]
    # mc.send_angles(np.degrees(q_pose).tolist(), spd)

def MoveCart(robt, spd, tool, wobj):
    pose_calc = wobj * robt.get_SE3() * tool.inv()

    # Convertir a formato [x, y, z, rx, ry, rz] en grados.
    pose = list(pose_calc.t) + list(pose_calc.rpy(order='zyx', unit='deg'))

    print(pose)

    # mc.send_coords(pose, spd, 1)

T = SE3(0.3, 0.1, 0.5) * SE3.OA([0, 1, 0], [0, 0, 1])  # Posición + orientación

TB=SE3([
       [ -1,   0,   0,   -103.6  ],
       [  0,  -1,   0,   -89.1   ],
       [  0,   0,   1,    331.4  ],
       [  0,   0,   0,   1       ]
       ])

# Crear robtarget
target = RobTarget(TB, [-1, -1, -1])

# print(target)                          # Por default: (x, y, z, rx, ry, rz) en grados


# wobj: en (100, 100, 100), z hacia abajo
# Z hacia abajo = rotación de 180° alrededor del eje X
wobj = SE3(100, 100, 100) * SE3.Rx(np.pi)

# tool: 150mm en Z (desde la brida hacia afuera)
tool = SE3(0, 0, 150)

# target.find_valid_configs(cobot, tool, wobj)

# MoveCart(target, 0, tool, wobj)
print(matrix_to_pose(target.pose))
MoveCart(target, 0, tool, wobj)
print(target)


[-103.6, -89.1, 331.4, 0.0, -0.0, 180.0]
[-3.5999999999999943, 189.1, -81.39999999999998, -180.0, -0.0, 180.0]
RobTarget(pose=SE3([-103.6, -89.1, 331.4], rpy=[0.0, -0.0, 180.0] deg), config=[-1, -1, -1])


In [2]:
from geometry_msgs.msg import TransformStamped, PoseStamped
from tf2_ros import TransformBroadcaster
import rclpy
from rclpy.node import Node
from spatialmath import SE3
from scipy.spatial.transform import Rotation as R

class TFPublisher(Node):
    def __init__(self):
        super().__init__('robtarget_tf_publisher')
        self.br = TransformBroadcaster(self)
        self.robtarget = SE3(100, 0, 200) * SE3.RPY([0, 0, 90], unit='deg')  # valor inicial
        self.timer = self.create_timer(0.1, self.publish_tf)

        # Subscriptor al tópico 'robtarget_pose'
        self.sub = self.create_subscription(
            PoseStamped,
            'robtarget_pose',
            self.pose_callback,
            10
        )

    def pose_callback(self, msg: PoseStamped):
        # Convertir PoseStamped → SE3
        t = msg.pose.position
        q = msg.pose.orientation
        rot = R.from_quat([q.x, q.y, q.z, q.w]).as_matrix()
        T = SE3(rot, [t.x * 1000, t.y * 1000, t.z * 1000])  # convertir a mm
        self.robtarget = T
        self.get_logger().info(f"RobTarget actualizado: {T.t.tolist()}")

    def publish_tf(self):
        if self.robtarget is None:
            return

        t = TransformStamped()
        t.header.stamp = self.get_clock().now().to_msg()
        t.header.frame_id = 'base'
        t.child_frame_id = 'robtarget1'

        pos = self.robtarget.t
        rot = self.robtarget.R
        quat = R.from_matrix(rot).as_quat()

        t.transform.translation.x = pos[0] / 1000.0
        t.transform.translation.y = pos[1] / 1000.0
        t.transform.translation.z = pos[2] / 1000.0
        t.transform.rotation.x = quat[0]
        t.transform.rotation.y = quat[1]
        t.transform.rotation.z = quat[2]
        t.transform.rotation.w = quat[3]

        self.br.sendTransform(t)

def main():
    # rclpy.init()
    # node = TFPublisher()
    # rclpy.spin(node)
    # node.destroy_node()
    # rclpy.shutdown()
    rclpy.init()
    node = TFPublisher()

    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        print("Interrumpido por el usuario")

    node.destroy_node()
    rclpy.shutdown()

if __name__ == '__main__':
    main()


Interrumpido por el usuario


RCLError: failed to shutdown: rcl_shutdown already called on the given context, at ./src/rcl/init.c:241

In [3]:
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import PoseStamped
from spatialmath import SE3
from scipy.spatial.transform import Rotation as R
import time

def main():
    rclpy.init()
    node = rclpy.create_node('robtarget_pose_publisher')
    pub = node.create_publisher(PoseStamped, 'robtarget_pose', 10)
    time.sleep(1)  # esperar a que arranque el subscriber

    # Pose en SE3
    T = SE3(0.4, 0.2, 0.5) * SE3.RPY([0, 45, 90], unit='deg')  # en metros

    pos = T.t
    rot = T.R
    quat = R.from_matrix(rot).as_quat()

    msg = PoseStamped()
    msg.header.frame_id = 'base'
    msg.pose.position.x = pos[0]
    msg.pose.position.y = pos[1]
    msg.pose.position.z = pos[2]
    msg.pose.orientation.x = quat[0]
    msg.pose.orientation.y = quat[1]
    msg.pose.orientation.z = quat[2]
    msg.pose.orientation.w = quat[3]

    pub.publish(msg)
    node.get_logger().info(f"Publicado robtarget_pose: {pos} | {quat}")
    rclpy.shutdown()

if __name__ == '__main__':
    main()


[INFO] [1748204495.911346831] [robtarget_pose_publisher]: Publicado robtarget_pose: [0.4 0.2 0.5] | [-0.27059805  0.27059805  0.65328148  0.65328148]
